In [1]:
import fastf1
from fastf1 import plotting
from fastf1.ergast import Ergast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fastf1.Cache.enable_cache("../f1_cache")

In [3]:
year = 2024
race_name = "Belgium"

In [4]:
race = fastf1.get_session(year, race_name, "R")
race.load()

core           INFO 	Loading data for Belgian Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

In [6]:
weather = race.weather_data
weather.groupby(["Rainfall"]).mean()

,Time,AirTemp,Humidity,Pressure,TrackTemp,WindDirection,WindSpeed
Rainfall,,,,,,,
False,0 days 01:08:14.308802919,21.328467,49.583942,978.09635,42.267153,140.985401,1.683212


In [8]:
track_miles = {
    "Spa": 4.352,
    "Baku": 3.73,
    "Silverstone": 3.66,
    "Sochi": 3.634,
    "Paul Ricard": 3.63,
    "Suzuka": 3.608,
    "Monza": 3.6,
    "Yas Marina": 3.451,
    "COTA": 3.426,
    "Shanghai": 3.387,
    "Bahrain": 3.363,
    "Melbourne": 3.295,
    "Singapore": 3.147,
    "Catalunya": 2.892,
    "Hockenheim": 2.842,
    "Hungaroring": 2.722,
    "Montreal": 2.71,
    "Red Bull Ring": 2.683,
    "Interlagos": 2.677,
    "Mexico City": 2.674,
    "Monaco": 2.074
}

track_km = {
    "Spa": 7.004,
    "Baku": 6.003,
    "Silverstone": 5.891,
    "Sochi": 5.848,
    "Paul Ricard": 5.841,
    "Suzuka": 5.807,
    "Monza": 5.793,
    "Yas Marina": 5.554,
    "COTA": 5.513,
    "Shanghai": 5.451,
    "Bahrain": 5.412,
    "Melbourne": 5.303,
    "Singapore": 5.065,
    "Catalunya": 4.655,
    "Hockenheim": 4.574,
    "Hungaroring": 4.381,
    "Montreal": 4.361,
    "Red Bull Ring": 4.318,
    "Interlagos": 4.309,
    "Mexico City": 4.304,
    "Monaco": 3.337
}

In [9]:
quali = fastf1.get_session(year, race_name, "Q")
quali.load()

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule
core           INFO 	Loading data for Belgian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '44', '4', '81', '63', '55', '14', '31', '23', '10', '3', '77', '18', '27', '20', '

In [17]:
laps = quali.laps.pick_driver("VER").pick_accurate()
driver_best = laps[~laps['PitOutTime'].notna() & ~laps['PitInTime'].notna() & laps['IsPersonalBest'] == True].pick_fastest()
quali_time = driver_best.LapTime.total_seconds()

/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


In [19]:
race_laps = race.laps.pick_driver("VER").pick_accurate()
race_laps = race_laps[~race_laps['PitOutTime'].notna() & ~race_laps['PitInTime'].notna()]

/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


In [30]:
quali_laps = quali.laps.pick_driver("VER").pick_accurate()
driver_best = quali_laps[~quali_laps['PitOutTime'].notna(
) & ~quali_laps['PitInTime'].notna() & quali_laps['IsPersonalBest'] == True].pick_fastest()
quali_time = driver_best.LapTime.total_seconds()

quali_time


/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


113.159

In [32]:
drv = "VER"
race_laps = race.laps.pick_driver(drv).pick_accurate()
race_laps = race_laps[~race_laps['PitOutTime'].notna(
) & ~race_laps['PitInTime'].notna()]
avg_lap_time = race_laps['LapTime'].mean().total_seconds()
compound_counts = race_laps['Compound'].value_counts(
).to_dict()
compound_used = max(compound_counts, key=compound_counts.get)
avg_sector_one = race_laps["Sector1Time"].dt.total_seconds(
).mean()
avg_sector_two = race_laps["Sector2Time"].dt.total_seconds(
).mean()
avg_sector_three = race_laps["Sector3Time"].dt.total_seconds(
).mean()
driver_team = race.get_driver(drv)['TeamName']

/opt/homebrew/lib/python3.11/site-packages/fastf1/core.py:3067: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


In [33]:
avg_sector_three

29.046538461538457

In [35]:
data = pd.read_csv("../data_store/race_quali_data.csv")
data.head()

,Year,Race,Driver,Team,QualiTime,RaceTimeAvg,Compound,Sector1Avg,Sector2Avg,Sector3Avg,AirTemp,TrackTemp,Humidity,WindSpeed,TrackLengthKM
0,2022,Belgium,1,Red Bull Racing,103.665,112.384750,MEDIUM,32.071028,49.834972,30.478750,21.787179,36.083974,40.519231,1.519231,7.004
1,2022,Belgium,55,Ferrari,104.297,113.030361,HARD,32.394111,50.024278,30.611972,21.787179,36.083974,40.519231,1.519231,7.004
2,2022,Belgium,11,Red Bull Racing,104.462,112.844444,MEDIUM,32.200389,50.053750,30.590306,21.787179,36.083974,40.519231,1.519231,7.004
3,2022,Belgium,16,Ferrari,104.551,113.913444,MEDIUM,32.284222,50.793306,30.835917,21.787179,36.083974,40.519231,1.519231,7.004
4,2022,Belgium,31,Alpine,105.180,114.182361,MEDIUM,32.234222,51.148861,30.799278,21.787179,36.083974,40.519231,1.519231,7.004
